# Midterm - Network Request Detection
Datasets from [Web Network Traffic dataset](https://www.kaggle.com/datasets/rudrakumar96/web-firewall-good-and-bad-request/data).

## Importing Packages

In [3]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report 

## Read in and Evaluate Data

In [18]:
base_path = '../datasets/network_requests/'
combine_paths = ['2bad_reqff.csv', '2good_reqff.csv']
testing = pd.read_csv(base_path + 'Testing_data.csv')

combined = pd.concat([pd.read_csv(base_path + f) for f in combine_paths], ignore_index=True)

In [19]:
combined.head()

,method,path,body,single_q,double_q,dashes,braces,spaces,percentages,semicolons,angle_brackets,special_chars,path_length,body_length,badwords_count,class
0,POST,/doLogin,uid=ZAP&passw=ZAP&btnSubmit=Login,0,0,0,0,0,0,0,0,0,8,33,1,bad
1,POST,/sendFeedback,cfile=comments.txt&name=ZAP&email_addr=ZAP&sub...,0,0,0,0,7,0,0,0,0,13,124,0,bad
2,GET,/admin/clients.xls,NaN,0,0,0,0,0,0,0,0,0,18,0,1,bad
3,GET,/my%20documents/JohnSmith/Bank%20Site%20Docume...,NaN,0,0,0,0,3,0,0,0,0,57,0,0,bad
4,GET,/my%20documents/JohnSmith/Bank%20Site%20Docume...,NaN,0,0,0,0,3,0,0,0,0,82,0,0,bad


In [23]:
combined.describe()

,single_q,double_q,dashes,braces,spaces,percentages,semicolons,angle_brackets,special_chars,path_length,body_length,badwords_count
count,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.0,5797.000000,5797.00000,5797.0,5797.000000,5797.000000,5797.000000
mean,0.795066,0.587373,0.183198,0.780231,4.159565,0.0,0.202691,0.27773,0.0,52.059686,51.558565,0.807314
std,1.787375,1.402760,0.427538,1.844078,6.078129,0.0,0.589890,0.91106,0.0,111.756514,116.982400,1.513214
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,13.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,0.000000,0.00000,0.0,33.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,7.000000,0.0,0.000000,0.00000,0.0,69.000000,86.000000,1.000000
max,12.000000,14.000000,2.000000,10.000000,40.000000,0.0,3.000000,8.00000,0.0,2162.000000,2224.000000,10.000000


In [21]:
combined.size

92752

In [25]:
# Find the count of nulls in each column
combined.isnull().sum(axis = 0)

method               0
path                 0
body              3396
single_q             0
double_q             0
dashes               0
braces               0
spaces               0
percentages          0
semicolons           0
angle_brackets       0
special_chars        0
path_length          0
body_length          0
badwords_count       0
class                0
dtype: int64

In [22]:
testing.size

1740